# Azure代码解释器工具规范

本示例将介绍配置和使用Azure代码解释器工具规范（由Azure动态会话提供支持）。


In [ ]:
%pip install llama-index
%pip install llama-index-llms-azure
%pip install llama-index-tools-azure-code-interpreter

In [ ]:
# 设置Azure OpenAI代理from llama_index.llms.azure_openai import AzureOpenAIapi_key = "your-azure-openai-api-key"  # 你的Azure OpenAI API密钥azure_endpoint = "your-azure-openai-endpoint"  # 你的Azure OpenAI端点api_version = "azure-api-version"  # Azure API版本

In [ ]:
# from llama_index import AzureCodeInterpreterToolSpecfrom llama_index.tools.azure_code_interpreter import (    AzureCodeInterpreterToolSpec,)# from llama_index import ReActAgentfrom llama_index.core.agent import ReActAgent# 使用pool_managment_endpoint创建AzureCodeInterpreterToolSpec，设置为会话管理端点# 设置local_save_path是可选的，但建议设置为一个路径，工具可以自动保存从Python代码输出生成的任何中间数据。azure_code_interpreter_spec = AzureCodeInterpreterToolSpec(    pool_managment_endpoint="your-pool-management-endpoint",    local_save_path="local-file-path-to-save-intermediate-data",)llm = AzureOpenAI(    model="gpt-35-turbo",    deployment_name="gpt-35-deploy",    api_key=api_key,    azure_endpoint=azure_endpoint,    api_version=api_version,)# 创建ReActAgent并注入AzureDynamicSessionsToolSpec中定义的工具agent = ReActAgent.from_tools(    azure_code_interpreter_spec.to_tool_list(), llm=llm, verbose=True)

In [ ]:
# 您可以直接使用代码解释器，而无需LLM代理。print(azure_code_interpreter_spec.code_interpreter("1+1"))

{'$id': '1', 'status': 'Success', 'stdout': '', 'stderr': '', 'result': 2, 'executionTimeInMilliseconds': 11}


In [ ]:
# 使用简单的回答测试代理程序，这些回答可以利用Python代码。 print(agent.chat("告诉我西雅图的当前时间。"))

Thought: To provide the current time in Seattle, I need to calculate it based on the current UTC time and adjust for Seattle's time zone, which is Pacific Daylight Time (PDT) during daylight saving time and Pacific Standard Time (PST) outside of daylight saving time. PDT is UTC-7, and PST is UTC-8. I can use the code interpreter tool to get the current UTC time and adjust it accordingly.
Action: code_interpreter
Action Input: {'python_code': "from datetime import datetime, timedelta; import pytz; utc_now = datetime.now(pytz.utc); seattle_time = utc_now.astimezone(pytz.timezone('America/Los_Angeles')); seattle_time.strftime('%Y-%m-%d %H:%M:%S %Z%z')"}
Observation: {'$id': '1', 'status': 'Success', 'stdout': '', 'stderr': '', 'result': '2024-05-04 13:54:09 PDT-0700', 'executionTimeInMilliseconds': 120}
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The current time in Seattle is 2024-05-04 13:54:09 PDT.
The current time in Seattle is 2

In [ ]:
# 上传一份在华盛顿雷德蒙德市的一天温度文件样本，并提出一个问题res = azure_code_interpreter_spec.upload_file(    local_file_path="./TemperatureData.csv")if len(res) != 0:    print(        agent.chat("找到我上传的文件中最高的温度。")    )

Thought: I need to use the list_files tool to get the metadata for the uploaded file, and then use python to read the file and find the highest temperature.
Action: list_files
Action Input: {}
Observation: [RemoteFileMetadata(filename='TemperatureData.csv', size_in_bytes=514, file_full_path='/mnt/data/TemperatureData.csv')]
Thought: I have the metadata for the file. I need to use python to read the file and find the highest temperature.
Action: code_interpreter
Action Input: {'python_code': "import csv\n\nwith open('/mnt/data/TemperatureData.csv', 'r') as f:\n    reader = csv.reader(f)\n    next(reader)\n    highest_temp = float('-inf')\n    for row in reader:\n        temp = float(row[1])\n        if temp > highest_temp:\n            highest_temp = temp\nprint(highest_temp)"}
Observation: {'$id': '1', 'status': 'Success', 'stdout': '12.4\n', 'stderr': '', 'result': '', 'executionTimeInMilliseconds': 26}
Thought: I have the highest temperature. I can answer the question.
Answer: The hi

In [ ]:
# 请求LLM根据上传的文件绘制图表。# 由于设置了local_save_path，图表数据将自动保存到local_save_path。print(    agent.chat(        "使用我上传的温度数据，创建一个温度曲线。"    ))

Thought: To proceed, I need to list the files in the session pool to identify the uploaded file containing the temperature data.
Action: list_files
Action Input: {}
Observation: [RemoteFileMetadata(filename='TemperatureData.csv', size_in_bytes=514, file_full_path='/mnt/data/TemperatureData.csv')]
Thought: The file containing the temperature data is identified as 'TemperatureData.csv'. I need to read this file to extract the temperature data for plotting the temperature curve.
Action: code_interpreter
Action Input: {'python_code': "import pandas as pd\n\ndf = pd.read_csv('/mnt/data/TemperatureData.csv')\nprint(df.head())"}
Observation: {'$id': '1', 'status': 'Success', 'stdout': '             Time  Temperature\n0  4/11/2024 0:00          6.5\n1  4/11/2024 1:00          5.2\n2  4/11/2024 2:00          4.8\n3  4/11/2024 3:00          5.1\n4  4/11/2024 4:00          5.7\n', 'stderr': '', 'result': '', 'executionTimeInMilliseconds': 2103}
Thought: The data has been successfully loaded and p

In [ ]:
# 请求LLM对样本温度文件进行修改print(    agent.chat(        "将温度数据按降序重新排列，并将其保存回原始的csv文件中。"    ))

Thought: To proceed, I need to first list the files in the session pool to identify the uploaded temperature data file.
Action: list_files
Action Input: {}
Observation: [RemoteFileMetadata(filename='TemperatureCurve.png', size_in_bytes=35512, file_full_path='/mnt/data/TemperatureCurve.png'), RemoteFileMetadata(filename='TemperatureData.csv', size_in_bytes=514, file_full_path='/mnt/data/TemperatureData.csv')]
Thought: The temperature data file is identified as 'TemperatureData.csv'. I need to read its contents, rearrange the temperature data in descending order, and then save the changes back to the original CSV file.
Action: code_interpreter
Action Input: {'python_code': "import pandas as pd\n\ndf = pd.read_csv('/mnt/data/TemperatureData.csv')\ndf_sorted = df.sort_values(by='Temperature', ascending=False)\ndf_sorted.to_csv('/mnt/data/TemperatureData.csv', index=False)\n'filename': 'TemperatureData.csv'"}
Observation: {'$id': '1', 'status': 'Failure', 'stdout': '', 'stderr': "illegal ta

In [ ]:
# 下载修改后的文件azure_code_interpreter_spec.download_file_to_local(    remote_file_path="TemperatureData.csv",    local_file_path="/.../SortedTemperatureData.csv",)

In [ ]:
# 用于比较，打印原始文件的前10行with open("/.../TemperatureData.csv", "r") as f:    for i in range(10):        print(f.readline().strip())

Time,Temperature
4/11/2024 0:00,6.5
4/11/2024 1:00,5.2
4/11/2024 2:00,4.8
4/11/2024 3:00,5.1
4/11/2024 4:00,5.7
4/11/2024 5:00,5.1
4/11/2024 6:00,4.5
4/11/2024 7:00,5.5
4/11/2024 8:00,5.3


In [ ]:
# 用于比较，打印从会话池下载的已排序文件的前10行with open("/.../SortedTemperatureData.csv", "r") as f:    for i in range(10):        print(f.readline().strip())

Time,Temperature
4/11/2024 20:00,12.4
4/11/2024 19:00,12.3
4/11/2024 17:00,12.3
4/11/2024 18:00,12.1
4/11/2024 16:00,11.7
4/11/2024 15:00,11.3
4/11/2024 21:00,10.9
4/11/2024 22:00,10.0
4/11/2024 23:00,9.4
